In [1]:
import copy

import pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tqdm
from scipy import signal
from scipy.signal import find_peaks_cwt
from matplotlib import rcParams
from statsmodels.graphics.tsaplots import plot_acf

In [15]:
class Analysis():
    def __init__(self,filePath,grid_lenth):
        self.filePath=filePath
        self.lenth=0
        self.datas=[]
        self.data_columns=[]
        self.hatching_points=[]
        self.onset_points=[]
        self.max_min_diff=[]
        self.grid_lenth=grid_lenth
        self.filtered_signals=[]
        self.total_movements=[]
        self.average_above_threshold=[]
        self.durations=[]
        self.moving_percentage=[]
        self.average_bout=[]

    def loading_datas(self):
        for each_file in os.listdir(self.filePath):
            if each_file.endswith(".csv"):
                current=pd.read_csv(self.filePath + each_file)
                self.datas.append(current)
                self.data_columns.append(current.columns)
        self.lenth =len(self.datas)

    def knn_norm(self,data, k=7, mode="max"):
        lenth = len(data)
        output_list = []
        for i in range(lenth - k):
            if mode == "max":
                output_list.append((np.max(data[i:i + k])))
            elif mode == "mean":
                output_list.append((np.mean(data[i:i + k])))
            elif mode == "min":
                output_list.append((np.min(data[i:i + k])))

        return np.hstack((np.ones(lenth-len(output_list))*output_list[0],np.array(output_list)))

    def max_min_run(self):
        for each_ in range(self.lenth):
            diffs=[]
            for j in tqdm.tqdm(range(self.grid_lenth)):
                normBright = np.array(self.datas[each_][self.data_columns[each_][j]])
                normBright_max = self.knn_norm(normBright, 60, "max")
                normBright_min = self.knn_norm(normBright, 60, "min")
                diff = normBright_max - normBright_min
                diffs.append(copy.deepcopy(diff))
            self.max_min_diff.append(diffs)


    def hatching_onset_detection(self):
        for each_ in range(self.lenth):
            onset_point=[]
            filtered_signals=[]
            for j in tqdm.tqdm(range(self.grid_lenth)):
                diff=self.max_min_diff[each_][j]
                cut= diff[0:5000]
                threshold= np.max(cut)
                onset_point.append(np.where(diff>threshold)[0][0])
                diff[np.where(diff<=threshold)[0]]=0
                diff[np.where(diff>threshold)[0][0]]=0
                filtered_signals.append(diff)
            self.onset_points.append(onset_point)
            self.filtered_signals.append(filtered_signals)

    def hatching_complete_detection(self):
        for each_ in range(self.lenth):
            maxss=[]
            for j in tqdm.tqdm(range(self.grid_lenth)):
                ndiff=self.knn_norm(self.max_min_diff[each_][j],8000,"mean")
                increase_point=np.where(ndiff>np.mean(ndiff)*1.2)[0][0]
                cut=ndiff[:increase_point]
                maxs =find_peaks_cwt(cut[10000:],500)
                maxs+=10000
                cuted=maxs[-2]
                maxss.append((maxs[-2],maxs[-1]))
                cut=cut[maxs[-2]:maxs[-1]]
                mins =find_peaks_cwt(cut**-1,80)
                predict=0
                reverse_min = mins[::-1]
                for i in range(len(mins)):
                    if i ==0:
                        continue
                    else:
                        if cut[reverse_min[i]] > cut[reverse_min[i-1]]:
                            predict=reverse_min[i-1]+maxs[-2]
                            #hatching_points.append(predict)
                            break
                if predict<(maxs[-2]+maxs[-1])/2:
                    self.hatching_points[each_].append(int((maxs[-1]+(maxs[-2]+maxs[-1])/2)/2))
                else:
                    self.hatching_points[each_].append(predict)

    def total_movement_quantity(self):
        self.total_movements=[]
        for each_ in range(self.lenth):
            Sum_above_threshold=[]
            for i in tqdm.tqdm(range(self.grid_lenth)):
                Sum_above_threshold.append(len(np.where(self.filtered_signals[each_][i][self.onset_points[each_][i]:self.hatching_points[each_][i]]!=0)[0]))
            self.total_movements.append(Sum_above_threshold)

    def average_movement_magnitude(self):
        for each_ in range(self.lenth):
            Average_of_values=[]
            for i in tqdm.tqdm(range(self.grid_lenth)):
                Average_of_values.append(np.average(self.datas[each_][self.data_columns[each_][i]][np.where(self.filtered_signals[each_][i][self.onset_points[each_][i]:self.hatching_points[each_][i]]!=0)[0]]))
            self.average_above_threshold.append(Average_of_values)

    def calculate_durations(self):
        for each_ in range(self.lenth):
            Duration=[]
            for i in tqdm.tqdm(range(self.grid_lenth)):
                Duration.append(len(self.filtered_signals[each_][i][self.onset_points[each_][i]:self.hatching_points[each_][i]]))
            self.durations.append(Duration)

    def calculate_moving_percentage(self):
        for each_ in range(self.lenth):
            Moving_Percentage=[]
            for i in tqdm.tqdm(range(self.grid_lenth)):
                Moving_Percentage.append(len(np.where(self.filtered_signals[each_][i][self.onset_points[each_][i]:self.hatching_points[each_][i]]!=0)[0])/len(self.filtered_signals[each_][i]))
            self.moving_percentage.append(Moving_Percentage)

    def calculate_average_bout(self):
        for each_ in range(self.lenth):
            Average_bout=[]
            for i in tqdm.tqdm(range(self.grid_lenth)):
                pos = np.where(self.filtered_signals[each_][i] > 0)[0]
                split = np.where(np.diff(pos) != 1)[0] + 1
                arr=np.split(self.filtered_signals[each_][i][pos],split)
                lenths=[]
                for j in arr:
                    lenths.append(len(j))
                Average_bout.append(np.mean(lenths))
            self.average_bout.append(Average_bout)




In [16]:
filePath = "../../data/"
#fileName = "output2022-04-27T15_16_04.csv"
fileName = "Raw1.csv"

In [20]:
test1=Analysis(filePath,24)
test1.loading_datas()
test1.max_min_run()
test1.hatching_onset_detection()

100%|██████████| 24/24 [00:00<00:00, 3425.09it/s]


In [21]:
test1.onset_points

[[12638,
  6325,
  10480,
  6251,
  8481,
  15360,
  9285,
  7129,
  5026,
  5494,
  7724,
  5623,
  5260,
  5331,
  11453,
  14128,
  6577,
  6595,
  7962,
  14035,
  11625,
  5685,
  9352,
  6959]]